In [1]:
import sqlite3
import pandas as pd
import os

# .db 파일들이 위치한 폴더 경로 (사용자의 실제 경로로 변경 필요)
folder_path = "C:/VISSIM_Workspace/network_test/network01.results"

# 폴더 내 모든 .db 파일 찾기
db_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".db")], key=lambda x: int(x.split(".")[0]))

# 분석할 수치 컬럼들
numeric_columns = ['ACCELERATION', 'DIST', 'LENGTH', 'VEHS', 'PERS',
                   'QUEUEDELAY', 'SPEEDAVGARITH', 'SPEEDAVGHARM', 'OCCUPRATE']

# 평균값을 저장할 리스트
all_data = []

# 각 데이터베이스에서 평균값 추출
for db_file in db_files:
    db_path = os.path.join(folder_path, db_file)

    # 데이터베이스 연결
    conn = sqlite3.connect(db_path)

    # 데이터 읽기
    query = f"SELECT {', '.join(numeric_columns)} FROM DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass"
    df = pd.read_sql_query(query, conn)

    # 연결 종료
    conn.close()

    # 컬럼별 평균값 계산
    avg_values = df.mean().to_dict()  # DataFrame → Dictionary 변환
    avg_values["DB_FILE"] = db_file  # 파일명 추가

    # 리스트에 저장
    all_data.append(avg_values)

# 최종 데이터프레임 생성
result_df = pd.DataFrame(all_data)

# 전체 평균값 (10개의 DB에 대한 평균)
final_avg = result_df.mean(numeric_only=True)

# 결과 출력
print("각 데이터베이스의 평균값:")
print(result_df)

print("\n10개 DB의 전체 평균값:")
print(final_avg)

# CSV 파일로 저장 (선택 사항)
result_df.to_csv("db_summary.csv", index=False)
final_avg.to_csv("db_overall_avg.csv", index=True)


각 데이터베이스의 평균값:
   ACCELERATION         DIST    LENGTH       VEHS       PERS  QUEUEDELAY  \
0      0.036772  7922.821868  5.289805  63.976667  63.976667   42.672229   
1      0.040065  7919.035944  5.282851  64.746111  64.746111   28.962803   
2      0.038015  7948.248456  5.275192  61.590833  61.590833   71.223777   
3      0.040183  7968.527892  5.261606  63.161944  63.161944   58.594637   
4      0.036161  7964.965486  5.316481  62.260556  62.260556   63.420654   
5      0.038834  7994.131009  5.260386  62.598333  62.598333   88.783816   
6      0.050804  7919.615173  5.222190  64.730000  64.730000   33.185602   
7      0.029185  7970.070249  5.263966  62.263889  62.263889   97.718866   
8      0.034428  7939.756246  5.250003  64.217500  64.217500   51.577341   
9      0.037952  7904.864518  5.330216  64.017222  64.017222   39.343446   

   SPEEDAVGARITH  SPEEDAVGHARM  OCCUPRATE DB_FILE  
0      91.197486     89.331444   0.235998    1.db  
1      92.991264     91.171618   0.226591   